## 사이킷런의 model_selection 모듈
1) 학습 데이터와 테스트 데이터 세트를 분리 </br>
2) 교차 검증 분할 및 평가
3) Estimator의 하이퍼 파라미터를 튜닝하기 위한 다양한 함수와 클래스 제공

In [2]:
# 학습/테스트 데이터 세트 분리 - train_test_split()
# 분리 안하면 어떤 문제가 생기는가?
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

iris=load_iris()
df_clf = DecisionTreeClassifier()
train_data = iris.data
train_label = iris.target
df_clf.fit(train_data, train_label)

# 학습 데이터 세트로 예측 수행
pred = df_clf.predict(train_data)
print('예측 정확도 : ', accuracy_score(train_label,pred))


예측 정확도 :  1.0


### train_test_split() 변수 설명
test_size : 전체 데이터에서 테스트 데이터 세트 크기를 얼마로 샘플링할 것인가를 결정. default는 0.25 </br>
train_size : 전체 데이터에서 학습용 데이터 세트 크기를 얼마로 샘플링할 것인가를 결정. test_size parameter를 통장적으로 사용해서 잘 사용 안함</br>
shuffle : 데이터를 분리하기 전에 데이터를 미리 섞을지 결정. default=True. 데이터 분산 -> 효율적인 학습 및 테스트 데이터 세트 생성</br>
random_state : 호출할 때마다 동일한 학습/테스트용 데이터 세트를 생성하기 위해 주어지는 난수 값. 지정 안하면 수행할 때마다 다른 학습/테스트용 데이터 생성</br>
반환값은 튜플

In [5]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

df_clf = DecisionTreeClassifier()
iris_data = load_iris()

X_train, X_test, y_train, y_test = train_test_split(iris_data.data, iris_data.target,test_size=0.3, random_state=121)

df_clf.fit(X_train, y_train)
pred = df_clf.predict(X_test)
print('예측 정확도 : {0:.4f}'.format(accuracy_score(y_test,pred)))

예측 정확도 : 0.9556


## 과적합 (overfitting)
모델이 학습 데이터에만 과도하게 최적화되어, 실제 예측을 다른 데이터로 수행했을 때 에측 성능이 과도하게 떨어지는 현상 </br>
solution : 교차 검증 </br>

## 교차 검증
본고사를 치르기 전 모의고사를 여러 번 치르는 것 </br>
별도의 여러 세트로 구성된 학습 데이터 세트와 검증 데이터 세트에서 학습과 평가를 수행하는 것 </br>
각 세트에서 수행한 평가 결과에 따라 하이퍼 파라미터 튜닝 등의 모델 최적화를 더욱 손쉽게 할 수 있음 </br>

### K 폴드 교차 검증
k개의 데이터 폴드 세트를 만들어서 k번 만큼 각 폴드 세트에서 학습과 검증 평가를 반복적으로 수행하는 방법

In [7]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
import numpy as np

iris=load_iris()
features = iris.data
label = iris.target
dt_clf = DecisionTreeClassifier(random_state=156)

# 5개의 폴드 세트로 분리하는 KFold 객체와 폴드 세트별 정확도를 담을 리스트 객체 생성
kfold = KFold(n_splits=5)
cv_accuracy = []
print('붓꽃 데이터 세트 크기 : ', features.shape[0])

붓꽃 데이터 세트 크기 :  150


In [8]:
n_iter=0

# KFold 객체의 split()을 호출하면 폴드별 학습용, 검증용 테스트의 로우 인덱스를 array로 반환
for train_index, test_index in kfold.split(features):
    # kfold.split()으로 반환된 인덱스를 이용해 학습용, 검증용 테스트 데이터 추출
    X_train, X_test = features[train_index], features[test_index]
    y_train, y_test = label[train_index], label[test_index]
    #학습 및 예측
    dt_clf.fit(X_train, y_train)
    pred=dt_clf.predict(X_test)
    n_iter+=1
    # 반복 시마다 정확도 측정
    accuracy = np.round(accuracy_score(y_test,pred),4)
    train_size = X_train.shape[0]
    test_size = X_test.shape[0]
    print('\n#{0} 교차 검증 정확도 : {1}, 학습 데이터 크기:{2}, 검증 데이터 크기:{3}'.format(n_iter, accuracy, train_size, test_size))
    print('#{0} 검증 세트 인덱스:{1}'.format(n_iter, test_index))
    cv_accuracy.append(accuracy)

# 개별 iteration별 정확도를 합하여 평균 정확도 계산
print('\n## 평균 검증 정확도 : ', np.mean(cv_accuracy))


#1 교차 검증 정확도 : 1.0, 학습 데이터 크기:120, 검증 데이터 크기:30
#1 검증 세트 인덱스:[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29]

#2 교차 검증 정확도 : 0.9667, 학습 데이터 크기:120, 검증 데이터 크기:30
#2 검증 세트 인덱스:[30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53
 54 55 56 57 58 59]

#3 교차 검증 정확도 : 0.8667, 학습 데이터 크기:120, 검증 데이터 크기:30
#3 검증 세트 인덱스:[60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83
 84 85 86 87 88 89]

#4 교차 검증 정확도 : 0.9333, 학습 데이터 크기:120, 검증 데이터 크기:30
#4 검증 세트 인덱스:[ 90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119]

#5 교차 검증 정확도 : 0.7333, 학습 데이터 크기:120, 검증 데이터 크기:30
#5 검증 세트 인덱스:[120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137
 138 139 140 141 142 143 144 145 146 147 148 149]

## 평균 검증 정확도 :  0.9


### Stratified K 폴드
불균형한(imbalanced) 분포도를 가진 레이블(결정 클래스) 데이터 집합을 위한 k폴드 방식 </br>
여기서 불균형이란? : 특정 레이블 값이 특이하게 많거나 매우 적어서 값의 분포가 한쪽으로 치우치는 것 </br>
ex) 정상대출:0, 대출사기:1 </br> 비록 건수는 작지만 알고리즘이 대출 사기를 예측하기 위한 중요한 피처 값을 가지고 있기 때문에 학습/테스트 세트에서 유지하도록 도와줌



In [ ]:
import pandas as pd

iris = load_iris()